In [14]:
import os

if "notebooks" in os.getcwd():
    os.chdir("..")

In [15]:
from tqdm.notebook import tqdm

In [16]:
from azure_cosmos_db import *

azure_cosmos = AzureCosmos()

azure_cosmos.list_containers()

[INFO] Cosmos client created


['eventBrite_events',
 'dice_events',
 'eventBrite_outputs',
 'shotgun_events',
 'eventBrite_links']

# Events, Dice & Shotgun Scraper

In [17]:
SCRAPER_NAME = "shotgun_events"
INPUT_FILE_PATH = "data/inputs/Purushottam POSH.xlsx"
SHEET_NAME = "Sheet4"

In [18]:
azure_cosmos.DATABASE_ID = "Scraper"
azure_cosmos.CONTAINER_NAME = SCRAPER_NAME

azure_cosmos.container = azure_cosmos.initialize_cosmosdb()

[INFO] Cosmos client created


In [19]:
# df = pd.read_csv(INPUT_FILE_PATH)
df = pd.read_excel(INPUT_FILE_PATH, sheet_name=SHEET_NAME)

df.head()

,Non Duplicates Organisers,Event Link,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,http://shotgun.live/venues/badaboum-club,https://shotgun.live/events/concert-baby-volca...,NaN,NaN,NaN
1,http://shotgun.live/venues/bungee,https://shotgun.live/events/stella-fiori-vultu...,NaN,NaN,NaN
2,http://shotgun.live/venues/glazart,https://shotgun.live/events/take-hit-x-glazart...,NaN,NaN,NaN
3,http://shotgun.live/venues/groovy-promo,https://shotgun.live/events/n-1-na-b-2-b-nujia...,NaN,NaN,NaN
4,http://shotgun.live/venues/jeudi-ok,https://shotgun.live/events/jeudi-ok-x-kidnapp...,NaN,NaN,NaN


In [20]:
urls = df["Non Duplicates Organisers"].to_list()

for url in tqdm(urls, desc=f"Uploading {len(urls)} URLs to {SCRAPER_NAME}"):
        print(url)
        data = {
            "id": hashlib.sha256(url.encode()).hexdigest(),
            "url": url,
            "processed": False,
            "sheet_name": SHEET_NAME
        }
        azure_cosmos.create_conversation(conversation_data=data)

Uploading 249 URLs to shotgun_events:   0%|          | 0/249 [00:00<?, ?it/s]

http://shotgun.live/venues/badaboum-club
[INFO] Conversation created successfully
http://shotgun.live/venues/bungee
[INFO] Conversation created successfully
http://shotgun.live/venues/glazart
[INFO] Conversation created successfully
http://shotgun.live/venues/groovy-promo
[INFO] Conversation created successfully
http://shotgun.live/venues/jeudi-ok
[INFO] Conversation created successfully
http://shotgun.live/venues/live-lisboa
[INFO] Conversation created successfully
http://shotgun.live/venues/mdk
[INFO] Conversation created successfully
http://shotgun.live/venues/nuits-sonores
[INFO] Conversation created successfully
http://shotgun.live/venues/ricardotavares
[INFO] Conversation created successfully
http://shotgun.live/venues/vibra-marketing-e-entretenimento
[INFO] Conversation created successfully
https://shotgun.live/venues
[INFO] Conversation created successfully
https://shotgun.live/venues/0-sm-0-se
[INFO] Conversation created successfully
https://shotgun.live/venues/00-10
[INFO] Co